# Título: Colaboración de Agentes para la creación de Artículos de prensa

## Introducción:
Eres un(a) profesional que trabaja para la BBC, cuya función es crear artículos de prensa. Te encanta tu trabajo pero a veces te falta inspiración para comenzar a bosquejar un nuevo artículo.

Un día vas tomarte un café con Malcolm Gladwell, le compartes tus pensamientos, a lo que él responde que existen muchas herramientas de Inteligencia Artifical que podrían ser ayuda para explorar ideas y acelerar el proceso creativo. A partir de esto te inscribes a un Workshop de IA Generativa para aprender sobre IA y sales con muchos conocimientos y ganas de utilizar AI en tu próximo proyecto.

## Objetivo:
Crear una solución de IA Generativa, que a partir de un prompt tipo texto, genere un bosquejo de tu próximo artículo, usando múltiples agentes (periodista y editor) que discutirán al respecto, generando varias versiones, hasta llegar a una versión final.

## Instrucciones:
Sigue los pasos detallados en este notebook para configurar el entorno, autenticarte en Google Cloud Platform, Configurar y coordina Agentes, Tareas, y Crews (tripulaciones) para crear artículos periodísticos.

### Instalamos dependencias

In [ ]:
# Instalamos la librería de vertex para python.
!pip install google-api-python-client

# Instalamos paquetes para conectarnos a los servicios de Google
!pip install google-auth
!pip install google-auth-oauthlib

# Instalamos crewAI, librería que nos ayuda a crear y gestionar Agentes
!pip install crewai
# Tools son mecanismos que pueden ser llamados por los agentes como una API, otros modelos de AI, etc.
!pip install 'crewai[tools]'

# Instalamos Vertex AI como proveedor del modelo que vamos a utilizar (tambien se puede usar Langchain para proveer otros modelos como OpenAI, Llama, etc.)
!pip install vertexai

# Crew AI admite varias formas de indicarle qué modelo queremos utilizar. Por el momento, la forma más fácil de pasarle un modelo a Crew AI es utilizando las clases estandarizadas de Langchain.
!pip install langchain-google-vertexai


### Autenticación Colab - GCP
Usamos la autenticación de Google Colab para conectarnos a GCP. Si no se está usando Colab, solo Python u otro runtime, se debe autenticar GCP a través de una cuenta de servicio.


In [2]:
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [3]:
# Configuramos los parametros de la API para que funcione con nuestro proyecto de Google Cloud
id_proyecto = "prj-uc-genai-labs"
region = "us-central1"

from langchain_google_vertexai import VertexAI
from langchain.embeddings import VertexAIEmbeddings

# Utilizando el paquete de langchain_google_genai solo podemos utilizar Gemini, pero con el paquete de vertexai tenemos acceso a todo el resto de modelos que GCP provee
# Incluyendo modelos open source desplegados a través de Model Garden
modeloGemini = VertexAI(model_name="gemini-1.5-pro-preview-0409", project=id_proyecto, location=region)


Una vez creada la instancia con la que los agentes llamarán al modelo de IA, debemos configurar los agentes necesarios para cada uno de nuestros pasos a definir. En este caso los pasos son:
1. A partir del prompt, se debe generar un bosquejo referencial del artículo (un periodista)
2. Una vez generado un primer bosquejo, este debe ser revisado con alguien con experiencia para dar el visto bueno (un editor)
3. Después de algunas iteraciones entre ambos agentes, se recibirá una versión revisada del bosquejo

Ahora, debemos configurar agentes que tomen distintos roles y ejecuten distintas acciones en esta cadena de producción

In [4]:
# Importamos Crew AI como librería que nos ayudará a crear y gestionar a los agentes
from crewai import Agent

# Creating a senior researcher agent with memory and verbose mode
periodista = Agent(
  role='Periodista',                          # Rol del agente
  goal='Generar un artículo sobre {topic}',   # Objetivo del rol
  verbose=True,                               # Define si se requiere cierto nivel de detalle para compartir información entre agentes, o simplemente dar el pase.
  memory=True,                                # Se utiliza para mantener memoria de ejecuciones anteriores
  llm=modeloGemini,                           # El modelo a utilizar
  backstory=(                                 # Describe la manera de actuar del agente mientras resuelve el objetivo
    "Eres una persona creativa y profesional"
    "llevas años trabajando haciendo artículos para los diarios más grandes del mundo,"
    "se te reconoce por tu estilo serio y pragmático"
  ),
  embedder=VertexAIEmbeddings,                # El embeding a utilizar para la memoria del agente
  allow_delegation=False,                     # Define la capacidad de un agente a delegar una tarea a otro agente en caso de que la tarea asignada no sea propia de su rol
  max_iter=6)                                 # Número de iteraciones máximas del agente para cada ejecución

editor = Agent(
  role='Editor',
  goal='Debe revisar y comentar puntos positivos y negativos de artículos periodisticos, cosas que haya que incluir o remover o cambiar. En este caso la tematica trata de {topic}',
  verbose=True,
  memory=True,
  llm=modeloGemini,
  backstory=(
    "Tienes mucha experiencia en el rubro periodistico"
    "y en tu cargo actual eres la persona que decide qué cosas saldrán al mercado y cuales no"
    "principalmente debido al nivel de autoridad que haz conseguido durante tu carrera"
  ),
  embedder=VertexAIEmbeddings,
  allow_delegation=False,
  max_iter=5
)

Una vez definidos los agentes debemos definir las tareas que estos estarán resolviendo

In [5]:
# Importamos la clase Task, que nos ayudará a crear las diferentes tareas, de una forma entendible para la IA
from crewai import Task

tarea_bosquejo = Task(
  description=(                                            # Describe la tarea
    "Crea el mejor posible con el tema de {topic}."
    "No olvides incluír partes drámaticas o amorosas, que al publico le gustan"
    "Comienza con una breve introducción, y luego ve describiendo en orden temporal de los sucesos."
    "Firma el articulo indicando tu nombre (inventa nombre y apellido) y la {fecha}."
  ),
  expected_output='Un borrador de artículo de periódico.', # Define el objeto que se espera como resultado
  agent=periodista,                                        # Asigna un Agente/Rol a la tarea
)

tarea_editor = Task(
  description=(
    "Debes revisar un artículo con la temática de {topic}."
    "Eres el encargado de dar el visto bueno, o de rechazar las obras de los periodistas según tu amplia experiencia"
    "tomando en cuenta de que el articulo ser interesante para el publico general, siempre incluyendo algún toque de drama, amor, polémica o hechos impresionantes."
  ),
  expected_output='La versión final del artículo',
  agent=editor,
)

Tenemos a los agentes y las tareas definidas. Ahora debemos juntarlos en una Crew (o Tripulación) y coordinarlos para que resuelvan nuestro objetivo final.

In [ ]:
from crewai import Crew, Process # Una Crew es una agrupación de agentes
from datetime import datetime
tripulacion = Crew(
  agents=[periodista, editor],
  tasks=[tarea_bosquejo, tarea_editor],
  process=Process.sequential,  # Optional: forma en que se coordinarán los agentes
  max_rpm=20,                  # Limite de requests por minuto
)

fecha = datetime.today()       # Creamos una fecha para el artículo

# Ejecutamos el flujo, mencionando las variables que definimos cuando creamos los agentes y sus tareas
resultado = tripulacion.kickoff(inputs={f'topic': 'La vida de la Reina Isabel', 'fecha': fecha}) # Acá podemos hacer correr el proceso e indicar la temática de la pelicula

# Imprimimos el resultado en la consola!
print(resultado)